# Goals and Mission

The goal of this file is to chose the best living location possible. The model includes following components.

Living location properties:
* Locations of interests.
    * Main comparison point is place of living.
    * Each individual is allowed unlimited number of places destinations of importance.
* Travel modes include, transit (PT), biking and walking.
* Fastest arrival times are provided for these area.
* Night and day travel (IK you are kinda freaky)

Goodness of fit:
* Each individual is assigned a single composite point.
    * They can divide it by however many locations they fancy and by whatever weighting fancied.
* The model will provide both individual and overall goodness scores for an area
* Weighting for night and day travel.

Enable the following :
* Maps JavaScript API
* Directions API

Get you google API key from the stored file.

# Navigator

Key google maps API and DB interactions are stored within the [Navigator](Navigator.py) class

In [1]:
from Navigator import Navigator
navigator = Navigator()

46 5.8 day


In [6]:
class Person():
    """Stores locations of interest for each individual"""
    def __init__(self):
        self.locations = {}
    def normalize(self):
        total = sum(self.locations.values())
        if total > 0:
            self.locations = {k: v / total for k, v in self.locations.items()}


Povilas = Person()
Povilas.locations = {"Imperial College London, South Kensington Campus":1,
                     "Imperial College London, White City Campus":1}
Povilas.normalize()

print(Povilas.locations)


{'Imperial College London, South Kensington Campus': 0.5, 'Imperial College London, White City Campus': 0.5}
